# More Commands, Command Loop Problems, a Java IDE

Now that we have the base `Command` class and a `CommandScheduler` we can really start talk about
programming for the robot, moving from this notebook environment to a real dvelopment environment, and
start learning about project structure and the wpilib.

Before we start, we need our `Command`, `CommandSceduler`, and `CountedCommand` classes. We will paste those in the cell below, and run that cell so we have these classes available to us.

In [2]:
// This is a Command class, initially we will just start with a constructor,
// naming, and an execute method.
abstract class Command
{
    private String m_name;
    
    Command()
    {
        // By default the name is the class name. If this is called
        // from the constructor of an extending class, then the
        // class name will be the name of the extending class.
        m_name = getClass().getName();
        m_name = m_name.substring(m_name.lastIndexOf('$') + 1);
    }
    
    void initialize()
    {
        
    }
    
    Command setName(String name)
    {
        m_name = name;
        return this;
    }
    
    String getName()
    {
        return m_name;
    }
    
    void execute()
    {
        //System.out.println("executing: " + m_name);
    }
    
    boolean isFinished()
    {
        return false;
    }
    
    void end(boolean interrupted)
    {
        
    }
}

class CountedCommand extends Command 
{
    // the step count
    private long m_stepCt = 0;
    // the number of steps we run before we stop
    final long m_numberOfSteps;
    
    CountedCommand(long numberOfSteps)
    {
        super();
        m_numberOfSteps = numberOfSteps;
    }
    
    @Override
    void initialize()
    {
        m_stepCt = 0;
    }
    
    
    @Override
    void execute()
    {
        m_stepCt += 1;
        System.out.println(getName()  + ":  stepCt: " + m_stepCt + " of " + m_numberOfSteps);
    }
    @Override
    boolean isFinished()
    {
        return m_stepCt >= m_numberOfSteps;
    }
}

// create and run a command to make sure it works
Command command = new CountedCommand(2).setName("test CountedCommand");
while (!command.isFinished()) {
    command.execute();
}

System.out.println("\nReset command:");
command.initialize();
while (!command.isFinished()) {
    command.execute();
}

class CommandScheduler
{
    final long RUN_TIME;
    final long STEP_TIME;
    
    // this is the list of commands to be executed the next time commands are run.
    private final List<Command> m_commandList = new ArrayList<>();
    // this is the list of commans waiting to be scheduled the next time commands are run.
    private final List<Command> m_toBeScheduled = new ArrayList<>();

    CommandScheduler(long runTime, long stepTime)
    {
        RUN_TIME = runTime;
        STEP_TIME = stepTime;
    }
    
    void schedule(Command command)
    {
        m_toBeScheduled.add(command);
    }
    
    void run()
    {
        // Setup the timing for running the commands in the loop.
        long startTime = System.currentTimeMillis();
        System.out.println("startTime = " + startTime);
        long endTime = startTime + RUN_TIME;
        long nextStepEnd = startTime;


        while (System.currentTimeMillis() < endTime)
        {
            // control of the step time
            long sleepTime = nextStepEnd - System.currentTimeMillis();
            if (sleepTime > 0)
            {
                try {
                    Thread.sleep(sleepTime);
                } catch(InterruptedException e) {
                    // OK, something is trying to kill this program,
                    // break out of the loop.
                    break;
                }
            }
            System.out.println("time = " + System.currentTimeMillis() + "; wanted to run at: " + nextStepEnd);
            nextStepEnd += STEP_TIME;
            
            // check the currently scheduled commands and remove any that have finished
            Iterator<Command> iter = m_commandList.iterator();
            while (iter.hasNext()) {
                Command command = iter.next();
                if (command.isFinished()) {
                    // remove the command we just got with the call to next()
                    iter.remove();
                    // call the end() method so the command can finish, the argument is false
                    // because the command was not interupted, but ended as expected.
                    command.end(false);
                }
            }
            
            // Adding any commands wating for scheduling
            iter = m_toBeScheduled.iterator();
            while (iter.hasNext()) {
            // get the next command to be scheduled
                Command command = iter.next();
                // remove it from the m_toBeScheduled list
                iter.remove();
                // initialize it
                command.initialize();
                // and add it to the scheduled command list
                m_commandList.add(command);
            }


            // run the commands
            iter = m_commandList.iterator();
            while (iter.hasNext()) 
            {
                iter.next().execute();
            }
        }
    }
}

CommandScheduler commandScheduler = new CommandScheduler(400, 40);
Command countedCommand4 = new CountedCommand(4);
commandScheduler.schedule(new CountedCommand(10).setName("cmd1"));
commandScheduler.schedule(new CountedCommand(5).setName("cmd2"));
commandScheduler.schedule(countedCommand4);
commandScheduler.run();


System.out.println("\nReschedule countedCommand4 and run the command scheduler");
commandScheduler.schedule(countedCommand4);
commandScheduler.run();

test CountedCommand:  stepCt: 1 of 2
test CountedCommand:  stepCt: 2 of 2

Reset command:
test CountedCommand:  stepCt: 1 of 2
test CountedCommand:  stepCt: 2 of 2
startTime = 1636001847972
time = 1636001847974; wanted to run at: 1636001847972
cmd1:  stepCt: 1 of 10
cmd2:  stepCt: 1 of 5
CountedCommand:  stepCt: 1 of 4
time = 1636001848016; wanted to run at: 1636001848012
cmd1:  stepCt: 2 of 10
cmd2:  stepCt: 2 of 5
CountedCommand:  stepCt: 2 of 4
time = 1636001848054; wanted to run at: 1636001848052
cmd1:  stepCt: 3 of 10
cmd2:  stepCt: 3 of 5
CountedCommand:  stepCt: 3 of 4
time = 1636001848094; wanted to run at: 1636001848092
cmd1:  stepCt: 4 of 10
cmd2:  stepCt: 4 of 5
CountedCommand:  stepCt: 4 of 4
time = 1636001848132; wanted to run at: 1636001848132
cmd1:  stepCt: 5 of 10
cmd2:  stepCt: 5 of 5
time = 1636001848177; wanted to run at: 1636001848172
cmd1:  stepCt: 6 of 10
time = 1636001848216; wanted to run at: 1636001848212
cmd1:  stepCt: 7 of 10
time = 1636001848253; wanted to r

# Making Some Cool New Commands

We have a pretty complete command infrastructure here. In this section we will talk about making a couple useful commands based on the `CountedCommand`. This will show you a little about implementing a bit of useful
functionality in a base class `CountedCommand` you can use many other places.

## Shooter Lift Command

This parallels our 2020-2021 season command to lift the ball into our shooter. The deal here is that we want to
enable the lift when the command is initialized (in the competition robot, this happened when a shoot button was pushed). The command extended the lift for some period of time (number of command steps) that it took the lift to reach full extension and deliver the ball to the shooter rollers - then the lift retracted so the next ball could position itself for lift. This is the code for a shooter lift that extends the `CountedCommand` class:

In [3]:
class ShooterLift extends CountedCommand
{
    boolean m_shooterLiftEnabled = false;
    
    ShooterLift(long numberOfSteps)
    {
        super(numberOfSteps);
    }
    
    @Override
    void initialize()
    {
        super.initialize();
        m_shooterLiftEnabled = true;
        // code to enable shooter lift subsystem
        System.out.println("Extend Lift");
    }
    
    @Override
    void end(boolean interupted)
    {
        m_shooterLiftEnabled = false;
        // code to disable shooter lift subsystem
        System.out.println("Retract Lift");
    }   
}

CommandScheduler commandScheduler = new CommandScheduler(400, 40);
commandScheduler.schedule(new ShooterLift(3));
commandScheduler.run();


startTime = 1636001849600
time = 1636001849600; wanted to run at: 1636001849600
Extend Lift
ShooterLift:  stepCt: 1 of 3
time = 1636001849645; wanted to run at: 1636001849640
ShooterLift:  stepCt: 2 of 3
time = 1636001849685; wanted to run at: 1636001849680
ShooterLift:  stepCt: 3 of 3
time = 1636001849720; wanted to run at: 1636001849720
Retract Lift
time = 1636001849760; wanted to run at: 1636001849760
time = 1636001849803; wanted to run at: 1636001849800
time = 1636001849845; wanted to run at: 1636001849840
time = 1636001849883; wanted to run at: 1636001849880
time = 1636001849921; wanted to run at: 1636001849920
time = 1636001849961; wanted to run at: 1636001849960
time = 1636001850002; wanted to run at: 1636001850000


OK, not much code - why is this so simple? `CountedCommand` does the heavy lifting here. It knows how to
run a command for some number of steps and we extend that functionality. In this case, the only things we
need to do with this command are:
* extend `initialize()` to enable the lift;
* extend `end(boolean interupted)` to disable the lift.


## Delayed Command

In [4]:
class DelayedCommand extends CountedCommand
{
    final Command m_nextCommand;
    final CommandScheduler m_commandScheduler;
    
    DelayedCommand(long numberOfSteps, Command command, CommandScheduler commandScheduler)
    {
        super(numberOfSteps);
        m_nextCommand = command;
        m_commandScheduler = commandScheduler;
    }
    
    @Override
    void end(boolean interupted)
    {
        m_commandScheduler.schedule(m_nextCommand);
        System.out.println("Scedule: " + m_nextCommand.getName());
    }
}

// Testn this command
CommandScheduler commandScheduler = new CommandScheduler(1000, 40);
commandScheduler.schedule(new DelayedCommand(6, new CountedCommand(3), commandScheduler));
commandScheduler.run();


startTime = 1636001852095
time = 1636001852095; wanted to run at: 1636001852095
DelayedCommand:  stepCt: 1 of 6
time = 1636001852139; wanted to run at: 1636001852135
DelayedCommand:  stepCt: 2 of 6
time = 1636001852180; wanted to run at: 1636001852175
DelayedCommand:  stepCt: 3 of 6
time = 1636001852217; wanted to run at: 1636001852215
DelayedCommand:  stepCt: 4 of 6
time = 1636001852255; wanted to run at: 1636001852255
DelayedCommand:  stepCt: 5 of 6
time = 1636001852298; wanted to run at: 1636001852295
DelayedCommand:  stepCt: 6 of 6
time = 1636001852338; wanted to run at: 1636001852335
Scedule: CountedCommand
CountedCommand:  stepCt: 1 of 3
time = 1636001852380; wanted to run at: 1636001852375
CountedCommand:  stepCt: 2 of 3
time = 1636001852417; wanted to run at: 1636001852415
CountedCommand:  stepCt: 3 of 3
time = 1636001852455; wanted to run at: 1636001852455
time = 1636001852495; wanted to run at: 1636001852495
time = 1636001852540; wanted to run at: 1636001852535
time = 1636001

Again, not much code as `CountedCommand` does the heavy again lifting here. It knows how to
run a command for some number of steps and we extend that functionality. In this case, the only things we
need to do with this command are:
* add a constructor to save the `Command` that should be delayed, and the `CommandScheduler` where
  the delayed command should be scheduled. NOTE: remember to call the `super()` to make sure the
  `CountedCommand` is correctly initialized;
* extend `end(boolean interupted)` to schedule the command.

## Using the `ShooterLift` and `DelayedCommand` Together

In last year's robot the shooter ball pen held 3 balls. In an ideal world there would be a single
button that activated 3 lifts with sufficient time between for the next ball top roll into position
on the lift. Here we will assume that 3 command cycles is good for shooting and 6 comman cycles is
good for waiting for the next ball to center on the lift, so we would activate the lift on command
cycles 0, 9, and 18:

In [5]:
CommandScheduler commandScheduler = new CommandScheduler(1000, 40);
ShooterLift shooterLift = new ShooterLift(3);
commandScheduler.schedule(shooterLift);
commandScheduler.schedule(new DelayedCommand(9, shooterLift, commandScheduler));
commandScheduler.schedule(new DelayedCommand(18, shooterLift, commandScheduler));
commandScheduler.run();

startTime = 1636001855256
time = 1636001855257; wanted to run at: 1636001855256
Extend Lift
ShooterLift:  stepCt: 1 of 3
DelayedCommand:  stepCt: 1 of 9
DelayedCommand:  stepCt: 1 of 18
time = 1636001855296; wanted to run at: 1636001855296
ShooterLift:  stepCt: 2 of 3
DelayedCommand:  stepCt: 2 of 9
DelayedCommand:  stepCt: 2 of 18
time = 1636001855336; wanted to run at: 1636001855336
ShooterLift:  stepCt: 3 of 3
DelayedCommand:  stepCt: 3 of 9
DelayedCommand:  stepCt: 3 of 18
time = 1636001855376; wanted to run at: 1636001855376
Retract Lift
DelayedCommand:  stepCt: 4 of 9
DelayedCommand:  stepCt: 4 of 18
time = 1636001855421; wanted to run at: 1636001855416
DelayedCommand:  stepCt: 5 of 9
DelayedCommand:  stepCt: 5 of 18
time = 1636001855457; wanted to run at: 1636001855456
DelayedCommand:  stepCt: 6 of 9
DelayedCommand:  stepCt: 6 of 18
time = 1636001855497; wanted to run at: 1636001855496
DelayedCommand:  stepCt: 7 of 9
DelayedCommand:  stepCt: 7 of 18
time = 1636001855536; wanted 

This idea of collecting commands is handled extensively by wpilib, see
[Command Groups](https://docs.wpilib.org/en/stable/docs/software/commandbased/command-groups.html)
for more information about how groups of commands can be packaged to build
really complex actions from a number of simple commands. We used this extensively
3 years ago to coordinate commands to 3 different subsystems to lift the robot from the floor to
a 20" high platform.

# What Happens if we put a Sleep or Loop in a Command?

This is something we really need to take a look at. In FTC block programming there is no command-based
structure. Programming is very linear. the robot generally performs one task until done, then moves
on to the next task. Each task usually contains a loop that is either looking at some sensor, like an
encoder, to decide when the task is done or running for some period of time.

The biggest source of problems I've seen with command-based programming is putting a loop like that inside
the `execute()` rather than using the encoder count or time as a contition for `isFinished()` to return `true` or `false`. This blocks the command-loop from running until that long running loop in the execute method runs.

Let's just see what that looks like by craating a `SleepCommand` and dropping it into the middle of
the 3-ball shoot we build in the last session: